<h1 align="center"> pyLDPC Tutorial: Basics </h1>

## update:  02/25/16 - v.0.7.0

<b><font color="red"> Since version 0.7: Coding and decoding functions take tG (Transposed G) instead of G, the coding matrix. Functions that construct it (CodingMatrix and CodingMatrix_systematic) return tG instead of G as well. </font></b> 

If it's the first time you're using pyldpc and you're looking for instructions to simulate a simple message transmission without too much details about how each module works, this tutorial is for <i> you</i>. Otherwise, I'd recommend <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/tree/master/pyLDPC-Presentation.ipynb?flush_cache=true"> pyLDPC Construction </a> (French, english coming soon) where you can find everything about construction of pyLDPC functions.

If you're familiar with coding and parity-check matrices, you may skip the theoretical summary in <i> 1- Mecanism </i> and move forward to <i> 2- Simulation </i>.

## 1-  Mecanism, Terminology:

First things first, let's make sure we're on the same page.

> <font color=#A44057> <h5> Coding </h5> </font>

In short, coding is adding to a binary array linear combinations of its original bits. Technically, it's a matrix multiplcation.
Let <font color=#04B404> G </font> be the <font color=#04B404> coding matrix shaped (k,n). </font>
The matricial product can be written tx = tG.tv for a more convenient "linear algebra" matricial product. Since v0.7, pyLDPC uses tG instead of G. 

<img src="Equations/Basics1.png">

This n-bits binary message <b><font color=#0101DF>x</font></b> is called a <i><b><font color=#0101DF> codeword </font></b></i> and will be transmitted through a noisy channel.

> <font color=#A44057> <h5> Channel model </h5> </font>

We'll consider AWGN <i> (Additive White Gaussian Noise) </i> as a model of the channel's noise. Intensity will be measured in SNR_{db} <i>(Signal Noise Ratio in decibels) </i> which is the commonly used unit in information theory. If you're more familiar with the variance of an AWGN: 

<img src="Equations/Basics2.png">

When passed through the noisy channel, the AWGN noise e is added to the n-bits BPSK modulated codeword x. 

Let <b><font color=#0101DF> y = x + e </font></b> the received n-bits message at the end of the channel. 

y is not binary, as far as we know it could be something like: 

<img src="Equations/Basics3.png">


> <font color=#A44057> <h5> Decoding </h5> </font>

if a  <b><font color=#0101DF>codeword x</font></b> is generated by G, the matrix H such as H.G' = 0 annuls x: H.x' = 0. 

<b><font color=#FF8000>H</font></b> is called <b><font color=#FF8000>Parity-check</font></b> matrix and is shaped (m,n).
Using H, decoding algorithms maximize probabilities [bit = 0, bit =1] conditional to {H.x' = 0}. 

<img src="Equations/Basics4.png">

Once we get back the <font color=#0101DF> codeword x </font>, finding the original k-bits message <font color=#04B404><b> v </b></font> is nothing but a linear system solving: v.G = x (G'v'=x'). 


## 2 -  Simulation Tutorial:

In practice, sucessful decoding relies on H's properties. Parity-check matrix has to be sparse (very few ones). Then coding matrix is constructed by solving H.tG = 0.

### Step 1: Construction of H
So far, pyLDPC constructs regular parity-check matrices using Gallager's method where H is constructed with the same number of ones per row, same number of ones per column. 

```python
function: RegularH
```

In [2]:
import pyldpc
import numpy as np

In [3]:
n = 15  # Number of columns
d_v = 4 # Number of ones per column, must be lower than d_c (because H must have more rows than columns)
d_c = 5 # Number of ones per row, must divide n (because if H has m rows: m*d_c = n*d_v (compute number of ones in H))

H = pyldpc.RegularH(n,d_v,d_c)

print("Regular parity-check matrix H({},{},{}):\n\n".format(n,d_v,d_c),H)

Regular parity-check matrix H(15,4,5):

 [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 1 0 0 1 0 0 1 0 0 1 0 1 0 0]
 [1 0 1 0 0 1 0 0 1 1 0 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 1 1 0 0 0 0 1 1 0]
 [0 0 0 0 1 0 0 0 1 1 1 1 0 0 0]
 [1 1 0 1 0 1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 1 0 1 0 1 1]
 [0 1 0 0 0 0 1 1 0 0 1 0 1 0 0]
 [1 0 1 1 1 0 0 0 1 0 0 0 0 0 0]]


### Step 3: Construction of G

#### The easy way: find any coding matrix G
The easy way to get a coding matrix G given H is using $CodingMatrix$.  The function returns tG, transposed G. 
One (skeptic persons in particular) can test H.tG = 0. 
```python
- CodingMatrix
```

In [4]:
tG = pyldpc.CodingMatrix(H)
print("Transposed Coding Matrix tG that goes with H above is:\n\n",tG)
print("\n With G,H you can code messages of {} bits into codewords of {} bits because G's shape is {}\n".format(tG.shape[1],
                                                                                           tG.shape[0],tG.T.shape))

Transposed Coding Matrix tG that goes with H above is:

 [[0 0 0 1 1 0]
 [0 1 1 1 0 1]
 [0 1 0 0 0 0]
 [1 1 0 0 0 1]
 [1 1 1 0 1 0]
 [1 0 1 0 1 1]
 [1 1 1 0 1 0]
 [1 0 1 1 0 0]
 [0 1 1 1 0 1]
 [1 0 0 0 0 0]
 [0 0 1 1 1 1]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]

 With G,H you can code messages of 6 bits into codewords of 15 bits because G's shape is (6, 15)



#### The hard way: find G in a systematic form
<img src="Equations/Basics5.png">

<b><font color=#04B404> The pros:</font></b>
- Keep track of the original bits even in the codeword (the first bits in y are therefore v + e).
- Many applications in images, text and sound transmission: we get to visualize the effect of AWGN on the original message and compare noisy messages to decoded ones.
- No need to solve tG.tv = tx after decoding since the first k bits of x are v. 

<b><font color=#DF0101> the cons: </font></b>
- Getting G in systematic form <u><b><font color=#DF0101> applies changes in H: columns permutations</font></b></u>.

Permutations in H's columns are not that bad, it doesn't change the codewords or the quality of decoding, but you have to pay attention and use the <u><b> new parity-check matrix H </b></u> instead of the first one.

The function 
```python
CodingMatrix_systematic 
``` 
takes one argument H and returns <b><u>modified H, transposed systematic tG </u></b>.

In [5]:
new_H,sys_tG = pyldpc.CodingMatrix_systematic(H) #If i'm willing to use only systematic form, I prefer to overwrite
                                                # H and simply compute: H,tG = pyldpc.CodingMatrix_systematic(H)
print("The new H is:\n\n",new_H)
print("\nSystematic tG is:\n\n",sys_tG)

### if you want to check if H.tG = 0:

print("\nBinary Product HxG' \n\n",pyldpc.BinaryProduct(H,tG))

The new H is:

 [[0 0 0 0 0 0 1 1 1 1 1 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 1 1 1 1 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 1 0 0 1 0 0 1 1]
 [1 1 0 0 0 0 1 0 1 0 0 1 0 0 0]
 [0 0 1 0 1 1 0 0 0 1 0 0 1 0 0]
 [0 0 0 1 1 0 0 0 1 0 0 0 1 1 0]
 [1 1 1 0 0 0 0 0 0 0 1 0 0 0 1]
 [0 0 0 0 0 1 1 1 0 1 0 1 0 0 0]
 [1 0 1 0 1 1 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0 1 1 1]
 [0 1 0 0 0 0 1 0 1 1 1 0 0 0 0]]

Systematic tG is:

 [[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]
 [0 0 0 1 1 0]
 [0 1 0 0 0 0]
 [0 1 1 1 0 1]
 [1 1 1 1 0 0]
 [1 1 0 1 1 1]
 [1 0 1 0 1 1]
 [1 1 0 1 1 1]
 [1 0 1 1 0 0]
 [0 0 1 1 1 1]]

Binary Product HxG' 

 [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


### Step 4:  Coding a k-bits message v 
G's shape is actually the tuple (message's length, codeword's length). 
The channel's noise paramater SNR is yours to choose. SNR around 8 db (decibels) corresponds to a standard-deviation around 0.4 .

To avoid confusions, let's fix them once and for all.

In [6]:
n,k = tG.shape
snr = 8
k,n

(6, 15)

Let's generate a random k-bits message:

In [13]:
v = np.random.randint(2,size=k)
print("The k-bits message v: \n",v)

The k-bits message v: 
 [0 0 1 1 1 1]


<b> Now CODING ! </b>

Here's the prototype of the Coding function 
```python
pyldpc.Coding(tG,v,snr)
    - tG: Transposed Coding Matrix
    - v: k-bits message 
    - snr: Signal to noise ratio
    
    returns: y
```

There is actually no limit for max_iter. Depending on SNR and matrices' shapes, you may need high max_iter. In practice, start with max_iter around 5 and increment it until your decoding is sucessful. For SNR > 6, max_iter = 15 is very convenient.

In [14]:
y = pyldpc.Coding(tG,v,snr)
print("The n-bits message received after transmission:\n\n",y)

The n-bits message received after transmission:

 [ 1.06457531 -0.5756515   0.53099215 -0.80319417  0.77876106 -1.17759391
  1.35828976  1.18763126 -0.75390809  1.07893604  1.00188166 -1.03680725
 -0.76774086 -0.94203675 -0.88122783]


### Step 5: Decoding 

Here's the prototype of the decoding functions. They represente the same algorithm. The second one uses logarithmic sum-products: he's faster. Unless you're interested in comparing them, you should use the full-log version.

```python
pyldpc.Decoding_BP(H,BitsAndNodes,y,snr,max_iter)
    - H: parity-check Matrix
    - y: n-bits noisy message
    - snr: Signal to noise ratio used in coding
    - max_iter: max number of decoding iterations 
    
    returns: codeword x 
```

```python
pyldpc.Decoding_logBP(H,BitsAndNodes,y,snr,max_iter)
    - H: parity-check Matrix
    - y: n-bits noisy message
    - snr: Signal to noise ratio used in coding
    - max_iter: max number of decoding iterations 
    
    returns: codeword x 
```


In [15]:
x_decoded = pyldpc.Decoding_logBP(H,y,snr,5)
print("The decoded n-bits codeword is:\n",x_decoded)


The decoded n-bits codeword is:
 [0 1 0 1 0 1 0 0 1 0 0 1 1 1 1]


You can check whether obtained x is a codeword or not by computing H.x'. 
if H.x' = 0, x is a codeword. 

In [16]:
print("H.x' = ",pyldpc.BinaryProduct(H,x_decoded))

H.x' =  [0 0 0 0 0 0 0 0 0 0 0 0]


### Step 6: Get original k-bits message v
Now let's finish our decoding and solve tG.tv = tx to find v, the original k-bits message.
```python
pyldpc.DecodedMessage(G,x)
    - tG: Transposed Coding Matrix
    - x: n-bits decoded message
    
    returns: k-bits original message
```

In [17]:
v_received = pyldpc.DecodedMessage(tG,x_decoded)
print("The k-bits decoded message v_recieved is:\n",v_received)
print("\nThe k-bits original message v is:\n",v)

The k-bits decoded message v_recieved is:
 [0 0 1 1 1 1]

The k-bits original message v is:
 [0 0 1 1 1 1]


<i> That's it ! </i>

## 3 - SNR, max_iter ... What  ? 

This section covers some statistics over LDPC performances. For several values of SNR, in a sample of 1000 messages, how many words are sucessfully decoded ? How do we choose max_iter in decoding ?


In [30]:
from time import time
n = 45  # Number of columns
d_v = 2 # Number of ones per column, must be lower than d_c (because H must have more rows than columns)
d_c = 3 # Number of ones per row, must divide n (because if H has m rows: m*d_c = n*d_v (compute number of ones in H))
H = pyldpc.RegularH(n,d_v,d_c)
tG = pyldpc.CodingMatrix(H)
k = tG.shape[1]
print("k,n = ",(k,n))

k,n =  (16, 45)


### SNR = 8 db [standard-deviation = 0.3]

In [31]:
sample_size = 1000
snr = 8
max_iter = 1
if 1:    
    matches=[]
    t = time()
    for i in range(sample_size): 
        v_sent = np.random.randint(2,size=k)
        y = pyldpc.Coding(tG,v_sent,snr) 
        x_decoded = pyldpc.Decoding_logBP(H,y,snr,max_iter)
        v_received = pyldpc.DecodedMessage(tG,x_decoded)
        matches.append((v_sent==v_received).all())
    t = time()-t
    print("Sucessful decoding rate for snr = {} and max_iter = {} is {}%".format(snr,max_iter,sum(matches)*100/sample_size))
    print("\nDecoding time of {} messages in seconds:".format(sample_size),t)

Sucessful decoding rate for snr = 8 and max_iter = 1 is 100.0%

Decoding time of 1000 messages in seconds: 4.181028842926025


Decoding is error-free, no need to change max_iter.

### SNR = 0 db [standard-deviation = 1]

In [32]:
sample_size = 1000
snr = 0
max_iter = 1
if 1:    
    matches=[]
    t = time()
    for i in range(sample_size): 
        v_sent = np.random.randint(2,size=k)
        y = pyldpc.Coding(tG,v_sent,snr) 
        x_decoded = pyldpc.Decoding_logBP(H,y,snr,max_iter)
        v_received = pyldpc.DecodedMessage(tG,x_decoded)
        matches.append((v_sent==v_received).all())
    t = time()-t
    print("Sucessful decoding rate for snr = {} and max_iter = {} is {}%".format(snr,max_iter,sum(matches)*100/sample_size))
    print("\nDecoding time of {} messages in seconds:".format(sample_size),t)

Sucessful decoding rate for snr = 0 and max_iter = 1 is 28.2%

Decoding time of 1000 messages in seconds: 4.681572914123535


If we increase max_iter, we'll certainly get better results:

In [33]:
sample_size = 1000
snr = 0
max_iter = 100
if 1:    
    matches=[]
    t = time()
    for i in range(sample_size): 
        v_sent = np.random.randint(2,size=k)
        y = pyldpc.Coding(tG,v_sent,snr) 
        x_decoded = pyldpc.Decoding_logBP(H,y,snr,max_iter)
        v_received = pyldpc.DecodedMessage(tG,x_decoded)
        matches.append((v_sent==v_received).all())
    t = time()-t
    print("Sucessful decoding rate for snr = {} and max_iter = {} is {}%".format(snr,max_iter,sum(matches)*100/sample_size))
    print("\nDecoding time of {} messages in seconds:".format(sample_size),t)

Sucessful decoding rate for snr = 0 and max_iter = 100 is 73.9%

Decoding time of 1000 messages in seconds: 23.639029026031494


## 4 - Do not forget:

I included several functions in pyLDPC that are not mentioned above. You might need one of them at some point:

```python
pyldpc.BinaryProduct(A,B) 
- A: Binary Matrix or scipy.sparse.csr_matrix*
- B: Binary Matrix/vector or scipy.sparse.csr_matrix*
  returns: binary matrix product of A and B

pyldpc.BinaryRank(A)
- A: Binary Matrix
  returns: Rank of A
  
pyldpc.InCode(H,x)
- H: Parity-check matrix
- x: n-bits array

returns: boolean, True if H.x = 0
```

* *Details about matrices construction <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/">here </a>*
## 5 - Other Tutorials:

### User's guide: 

- LDPC Applications in Images transmission: <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Images.ipynb?flush_cache=true/"> User's guide: pyLDPC-Images Tutorial</a>

- LDPC Applications in Sound transmission: <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Sound.ipynb?flush_cache=true/"> User's guide: pyLDPC-Sound Tutorial</a>

- How do I chose my matrices in LDPC applications ? <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Tutorial-Matrices.ipynb?flush_cache=true/"> User's guide: pyLDPC-Matrices Tutorial</a>

### pyLDPC construction details:

- pyLDPC Construction Basics (French) : <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Presentation.ipynb?flush_cache=true/"> Construction details: pyLDPC </a>

- pyLDPC Images submodule Construction : <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Images-Construction.ipynb?flush_cache=true/"> LDPC-Images Construction </a>

- pyLDPC Sound submodule Construction : <a href="http://nbviewer.jupyter.org/github/janatiH/pyldpc/blob/master/pyLDPC-Sound-Construction.ipynb?flush_cache=true/"> LDPC-Sound Construction </a>
